In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from joblib import dump, load
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

C:\Users\Jesse\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [24]:
data_path = 'C:/Users/Jesse/Desktop/midterm_data/data/'
flights_df = pd.read_csv(data_path + 'flights.csv')
# flights = pd.read_csv(data_path+'flights_test.csv')

C:\Users\Jesse\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# columns we are allowed to use
cols = ['fl_date', 
        'mkt_unique_carrier', 
        'branded_code_share', 
        'mkt_carrier', 
        'mkt_carrier_fl_num', 
        'op_unique_carrier', 
       'tail_num', 
       'op_carrier_fl_num', 
       'origin_airport_id', 
       'origin', 
       'origin_city_name', 
       'dest_airport_id', 
       'dest', 
       'dest_city_name', 
       'crs_dep_time', 
       'crs_arr_time', 
       'dup', 
       'crs_elapsed_time', 
       'flights', 
       'distance', 
       'arr_delay']


In [12]:
# Hub airports dictionary we will use to create a new df
hub_airports = {
    'AS': ['SEA', 'ANC', 'LAX', 'PDX', 'SFO'],
    'AA': ['DFW', 'CLT', 'ORD', 'LAX', 'MIA', 'JFK', 'LGA', 'PHL', 'PHX', 'DCA'],
    'DL': ['ATL', 'BOS', 'DTW', 'LAX', 'MSP', 'JFK', 'LGA'],
    'UA': ['ORD', 'DEN', 'IAH', 'LAX'],
    'HA': ['HNL'],
    'F9': ['DEN', 'DEN', 'ORD'],
    'B6': ['JFK', 'BOS'],
    'WN': ['ATL', 'BWI', 'MDW'],
    'NK': ['ACY']
}

In [11]:
# n=10000
# flights = flights_df.loc[flights_df['arr_delay'].notna(), cols].sample(n=n)
# flights = flights_df.loc[flights_df['arr_delay'].notna(), cols]
# flights.shape

(15615741, 21)

In [12]:
# new column categories, morning, afternoon, evening
# identify whether airline is operating out of hub aiport remote airport
# bin the airports on size and how many flights take off from it
# Airline fleetsize

In [13]:
# drop columns

flights.drop(['mkt_unique_carrier', 
              'origin_city_name',
              'dest_city_name', 
              'dup', 
              'origin_airport_id', 
              'origin_city_name', 
              'dest_airport_id', 
              'dest_city_name',
              'flights'], axis=1, inplace=True)

In [16]:
# Fill na values

flights['crs_elapsed_time'].fillna(flights['crs_elapsed_time'].median(), inplace=True)
flights['tail_num'].fillna(0, inplace=True)
flights.isna().sum()

fl_date               0
branded_code_share    0
mkt_carrier           0
mkt_carrier_fl_num    0
op_unique_carrier     0
tail_num              0
op_carrier_fl_num     0
origin                0
dest                  0
crs_dep_time          0
crs_arr_time          0
crs_elapsed_time      0
distance              0
dtype: int64

In [17]:
# change fl_date to datetime format
flights['fl_date'] = pd.to_datetime(flights['fl_date'], format='%Y-%m-%d')
flights['dayofyear'] = flights['fl_date'].dt.dayofyear
flights['dayofweek'] = flights['fl_date'].dt.dayofweek
flights['month'] = flights['fl_date'].dt.month

# flights['day'] = flights['fl_date'].dt.day
# flights.drop('fl_date', axis=1, inplace=True)

In [18]:
# convert the crs_dep_time and crs_arr_time to string so it can be properly formated
def convert_time_to_string(x):
        if np.isnan(x):
            return '00:00'

        if x == 2400:
            return '23:59'

        string = str(int(x))

        if len(string) < 4:
            for _ in range(4-len(string)):
                string = '0' + string

        return string[:2] + ':' + string[2:]

In [19]:
# create new minute and hour column for each scheduled departure and arrival
flights['crs_dep_time'] = flights['crs_dep_time'].apply(lambda x: convert_time_to_string(x))
flights['crs_arr_time'] = flights['crs_arr_time'].apply(lambda x: convert_time_to_string(x))

# Convert to datetime
flights['crs_dep_time'] = pd.to_datetime(flights['crs_dep_time'], format='%H:%M')  
flights['crs_arr_time'] = pd.to_datetime(flights['crs_arr_time'], format='%H:%M')



In [20]:
# Create new columns for departing hr and departing min
flights['crs_dep_hr'] = flights['crs_dep_time'].dt.hour
flights['crs_dep_min'] = flights['crs_dep_time'].dt.minute

# create new 
flights['crs_arr_hr'] = flights['crs_arr_time'].dt.hour
flights['crs_arr_min'] = flights['crs_arr_time'].dt.minute

flights.drop(['crs_dep_time', 'crs_arr_time'], axis=1, inplace=True)

In [21]:
def get_part_of_day(x):
    if x == 0:
        return 'midnight'
    elif x > 0 and x<= 5:
        return 'early_morning'
    elif x > 5 and x<= 10:
        return 'morning'
    elif x > 10 and x <= 15:
        return 'noon'
    elif x > 15 and x <= 18:
        return 'afternoon'
    elif x > 18 and x<=21:
        return 'evening'
    elif x>21:
        return 'late evening'

# Convert to part of day
flights['dep_time_day'] = flights['crs_dep_hr'].apply(lambda x: get_part_of_day(x))
flights['arr_time_day'] = flights['crs_arr_hr'].apply(lambda x: get_part_of_day(x))

In [22]:
# Check if airport origin airport in a hub airport?


def is_hub(x):
    if x['mkt_carrier'] in hub_airports:
        if x['origin'] in hub_airports[x['mkt_carrier']]:
            return 1
        else:
            return 0
    else:
        return 0

    
flights['at_hub'] = flights.apply(lambda row: is_hub(row), axis=1)

In [25]:
# Get airports and categorize by how many flights take off from origin
highdep = flights_df.groupby('origin')['fl_date'].count().sort_values(ascending=False).index[:50]
medium_highdep = flights_df.groupby('origin')['fl_date'].count().sort_values(ascending=False).index[50:150]
mediumdep = flights_df.groupby('origin')['fl_date'].count().sort_values(ascending=False).index[150:250]    
lowdep = flights_df.groupby('origin')['fl_date'].count().sort_values(ascending=False).index[250:]

busy_dep = {'high': highdep,
        'medium-high': medium_highdep,
        'medium': mediumdep,
        'low': lowdep}

def get_busy(airport):
    for key, lists in busy_dep.items():
        if airport in lists:
            return key
        
    return 'low'


# For destination airports
high = flights_df.groupby('dest')['fl_date'].count().sort_values(ascending=False).index[:50]
medium_high = flights_df.groupby('dest')['fl_date'].count().sort_values(ascending=False).index[50:150]
medium = flights_df.groupby('dest')['fl_date'].count().sort_values(ascending=False).index[150:250]    
low = flights_df.groupby('dest')['fl_date'].count().sort_values(ascending=False).index[250:]

busy = {'high': high,
        'medium-high': medium_high,
        'medium': medium,
        'low': low}

def get_busy_dept(airport):
    for key, lists in busy.items():
        if airport in lists:
            return key
        
    return 'low'
    
    
flights['origin_traffic'] = flights['origin'].apply(lambda x: get_busy(x))
flights['departure_traffic'] = flights['dest'].apply(lambda x: get_busy_dept(x))

In [6]:
# flights = pd.read_csv('C:/Users/Jesse/Desktop/midterm_data/data/regression_df.csv')
#historical_arr_delay, historical_dep_delay, historical_arr_delay_std, historical_arr_delay_75Q, historical_arr_delay_25Q
# flights.drop(['historical_dep_delay', 'historical_arr_delay', 'historical_arr_delay_std', 'historical_arr_delay_75Q', 'historical_arr_delay_25Q'], axis=1, inplace=True)

In [26]:
# Using historical data for arrival delay
history = dict(flights.groupby(['origin', 'dest', 'mkt_carrier', 'month'])['arr_delay'].median())

# Create a new column
def get_historical_arr_delay(row):
    if (not pd.isnull(row['dest'])) and (not pd.isnull(row['origin'])) and (not pd.isnull(row['mkt_carrier'])) and (not pd.isnull(row['month'])):
        mean_arr_delay = history[(row['origin'], row['dest'], row['mkt_carrier'], row['month'])]
        return mean_arr_delay
    else:
        return 0

flights['historical_delay'] = flights.apply(lambda row: get_historical_arr_delay(row), axis=1)


KeyError: 'Column not found: arr_delay'

In [9]:
# flights.to_csv('C:/Users/Jesse/Desktop/midterm_data/data/regression_df2.csv', index=False)

In [23]:
# # Using historical data for departure delat
# flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format='%Y-%m-%d')
# flights_df['month'] = flights_df['fl_date'].dt.month

# history = dict(flights_df.groupby(['origin', 'mkt_carrier', 'month'])['dep_delay'].median())

# # Create a new column
# def get_historical_dep_delay(row):
#     if (not pd.isnull(row['origin'])) and (not pd.isnull(row['mkt_carrier'])) and (not pd.isnull(row['month'])):
#         mean_arr_delay = history[(row['origin'], row['mkt_carrier'], row['month'])]
#         return mean_arr_delay
#     else:
#         return 0

# flights['historical_dep_delay'] = flights.apply(lambda row: get_historical_dep_delay(row), axis=1)

In [11]:
# Using historical data for arr_delay std

# flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format='%Y-%m-%d')
# flights_df['month'] = flights_df['fl_date'].dt.month

history = dict(flights.groupby(['origin', 'dest', 'mkt_carrier', 'month'])['arr_delay'].std())

# Create a new column
def get_historical_arr_delay_std(row):
    if (not pd.isnull(row['dest'])) and (not pd.isnull(row['origin'])) and (not pd.isnull(row['mkt_carrier'])) and (not pd.isnull(row['month'])):
        std_arr_delay = history[(row['origin'], row['dest'], row['mkt_carrier'], row['month'])]
        return std_arr_delay
    else:
        return 0

flights['historical_delay_std'] = flights.apply(lambda row: get_historical_arr_delay_std(row), axis=1)

In [14]:
# save
flights.to_csv('C:/Users/Jesse/Desktop/midterm_data/data/regression_df2.csv', index=False)

In [18]:
# Using historical data for 75th quantile

# flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format='%Y-%m-%d')
# flights_df['month'] = flights_df['fl_date'].dt.month

history = dict(flights.groupby(['origin', 'dest', 'mkt_carrier', 'month'])['arr_delay'].quantile(0.75))

# Create a new column
def get_historical_arr_delay_75Q(row):
    if (not pd.isnull(row['dest'])) and (not pd.isnull(row['origin'])) and (not pd.isnull(row['mkt_carrier'])) and (not pd.isnull(row['month'])):
        Q75_arr_delay = history[(row['origin'], row['dest'], row['mkt_carrier'], row['month'])]
        return Q75_arr_delay
    else:
        return 0

flights['historical_delay_75Q'] = flights.apply(lambda row: get_historical_arr_delay_75Q(row), axis=1)

In [25]:
# Using historical data for 25th quantile

# flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format='%Y-%m-%d')
# flights_df['month'] = flights_df['fl_date'].dt.month

history = dict(flights.groupby(['origin', 'dest', 'mkt_carrier', 'month'])['arr_delay'].quantile(0.25))

# Create a new column
def get_historical_arr_delay_25Q(row):
    if (not pd.isnull(row['dest'])) and (not pd.isnull(row['origin'])) and (not pd.isnull(row['mkt_carrier'])) and (not pd.isnull(row['month'])):
        Q25_arr_delay = history[(row['origin'], row['dest'], row['mkt_carrier'], row['month'])]
        return Q25_arr_delay
    else:
        return 0

flights['historical_delay_25Q'] = flights.apply(lambda row: get_historical_arr_delay_25Q(row), axis=1)

In [35]:
history

{('ABE', 'ATL', 'DL', 1): -17.0,
 ('ABE', 'ATL', 'DL', 2): -17.0,
 ('ABE', 'ATL', 'DL', 3): -17.0,
 ('ABE', 'ATL', 'DL', 4): -17.0,
 ('ABE', 'ATL', 'DL', 5): -18.0,
 ('ABE', 'ATL', 'DL', 6): -16.0,
 ('ABE', 'ATL', 'DL', 7): -16.0,
 ('ABE', 'ATL', 'DL', 8): -14.0,
 ('ABE', 'ATL', 'DL', 9): -18.0,
 ('ABE', 'ATL', 'DL', 10): -12.0,
 ('ABE', 'ATL', 'DL', 11): -18.0,
 ('ABE', 'ATL', 'DL', 12): -22.0,
 ('ABE', 'BNA', 'G4', 5): -16.0,
 ('ABE', 'BNA', 'G4', 6): -15.75,
 ('ABE', 'BNA', 'G4', 7): -21.0,
 ('ABE', 'BNA', 'G4', 8): -26.0,
 ('ABE', 'BNA', 'G4', 9): -29.0,
 ('ABE', 'BNA', 'G4', 10): -4.25,
 ('ABE', 'BNA', 'G4', 11): -13.5,
 ('ABE', 'BNA', 'G4', 12): -12.0,
 ('ABE', 'CLT', 'AA', 1): -11.5,
 ('ABE', 'CLT', 'AA', 2): -5.5,
 ('ABE', 'CLT', 'AA', 3): -11.25,
 ('ABE', 'CLT', 'AA', 4): -13.0,
 ('ABE', 'CLT', 'AA', 5): -17.0,
 ('ABE', 'CLT', 'AA', 6): -15.0,
 ('ABE', 'CLT', 'AA', 7): -13.0,
 ('ABE', 'CLT', 'AA', 8): -14.25,
 ('ABE', 'CLT', 'AA', 9): -17.25,
 ('ABE', 'CLT', 'AA', 10): -16.0,


In [29]:
# # This dataframe has the historical_arr_delay
flights = pd.read_csv('C:/Users/Jesse/Desktop/midterm_data/data/regression_df.csv')


In [30]:
# drop brand_code_share
flights.drop(['branded_code_share', 'op_unique_carrier', ], axis=1, inplace=True)

In [31]:
X = flights.drop('arr_delay', axis=1)
y = flights['arr_delay']

In [32]:
# label encode
    # tail num DONE
    # origin DONE
    # dest DONE
    # mkt_carrier DONE 

# one hot encode
    # dep_time_day
    # arr_time_day
    # origin_traffic 
    # departure_traffic
    

# Standard Scalar results

In [33]:
# Label encode the origin airports and destination airpots
tail_num_le = LabelEncoder()
X['tail_num'] = tail_num_le.fit_transform(X['tail_num'])

# Label encode origin and destination
airports_le = LabelEncoder()
X['origin'] = airports_le.fit_transform(X['origin'])
X['dest'] = airports_le.transform(X['dest'])

# label encode mkt_carrier
mkt_carrier_le = LabelEncoder()
X['mkt_carrier'] = mkt_carrier_le.fit_transform(X['mkt_carrier'])

In [34]:
# Converting selectedcolumns and one hot encoding them
ohe_columns = ['dep_time_day', 'arr_time_day', 'origin_traffic', 'departure_traffic']

ct = ColumnTransformer(transformers=[('ohe', OneHotEncoder(), ohe_columns)], remainder='passthrough')

X_transformed= ct.fit_transform(X)

In [258]:
X_transformed_sample = X_transformed[:1000]
y_sample = y[:1000]

In [36]:
number_of_rows = X_transformed.shape[0]
random_indices = np.random.choice(number_of_rows, size=100000, replace=False)
X_transformed_sample = X_transformed[random_indices, :]

number_of_rows = y.shape[0]
y_sample = y[random_indices]


X_transformed_sample.shape, y_sample.shape

((100000, 43), (100000,))

In [37]:
xtrain, xtest, ytrain, ytest = train_test_split(X_transformed_sample, y_sample, test_size=0.2)

In [38]:
xg_reg = XGBRegressor()

xg_reg.fit(xtrain, ytrain)
xg_reg.score(xtest, ytest)

[15:09:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.03639984040215127

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'subsample': [0.7, 0.8, 0.9]
}

rs = RandomizedSearchCV(estimator=XGBRegressor(), param_distributions=param_grid, cv=2, verbose=True, n_iter=4)

rs.fit(xtrain, ytrain)

print(rs.best_score_)
print(rs.best_estimator_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[20:10:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[20:12:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [30]:
print(rs.best_score_)
print(rs.best_estimator_)

0.026746473220284395
XGBRegressor(colsample_bytree=0.7, max_depth=20, n_estimators=700,
             reg_alpha=1.1, reg_lambda=1.2, subsample=0.9)


In [ ]:
XGBRegressor(colsample_bytree=0.7, max_depth=20, n_estimators=700,
             reg_alpha=1.1, reg_lambda=1.2, subsample=0.9)

In [13]:
flights[['dayofyear', 
         'dayofweek', 
         'month', 
         'crs_dep_hr', 
         'crs_dep_min', 
         'crs_arr_hr', 
         'crs_arr_min', 
         'dep_time_day', 
         'arr_time_day', 
         'at_hub', 
         'origin_traffic', 
         'departure_traffic', 
         'historical_dep_delay', 
         'historical_arr_delay', 
         'historical_arr_delay_std', 
         'historical_arr_delay_75Q',
         'historical_arr_delay_25Q']].head(10)

,dayofyear,dayofweek,month,crs_dep_hr,crs_dep_min,crs_arr_hr,crs_arr_min,dep_time_day,arr_time_day,at_hub,origin_traffic,departure_traffic,historical_dep_delay,historical_arr_delay,historical_arr_delay_std,historical_arr_delay_75Q,historical_arr_delay_25Q
0,234,3,8,15,1,16,56,noon,afternoon,0,high,high,29.743056,18.762984,66.646439,26.0,-16.00
1,234,3,8,21,50,1,9,evening,early_morning,1,high,high,19.266934,5.736611,49.393412,15.0,-21.00
2,234,3,8,23,50,8,33,late evening,morning,0,high,high,16.574803,18.762984,66.646439,26.0,-16.00
3,234,3,8,18,14,20,0,afternoon,evening,1,high,high,19.266934,21.650996,64.482387,28.0,-14.00
4,234,3,8,6,18,9,0,morning,morning,0,high,high,34.494299,24.717784,81.432011,31.0,-17.00
5,234,3,8,13,53,17,4,noon,afternoon,0,high,high,32.489899,18.762984,66.646439,26.0,-16.00
6,234,3,8,18,36,21,19,afternoon,evening,1,high,high,19.266934,15.004219,52.146022,26.0,-15.00
7,234,3,8,22,29,4,19,late evening,early_morning,0,high,high,26.144681,18.762984,66.646439,26.0,-16.00
8,234,3,8,5,45,8,21,early_morning,morning,0,high,high,25.621849,18.762984,66.646439,26.0,-16.00
9,234,3,8,20,0,22,53,evening,late evening,1,high,high,19.266934,12.963087,58.677028,17.0,-17.25


In [7]:
flights_sampled = flights.sample(100000)

In [1]:
# train = flights_sampled.sample(frac=0.5)
# test = flights_sampled.loc[~flights_sampled.index.isin(train.index)]
# multinomModel = smf.ols('arr_delay ~branded_code_share+mkt_carrier+mkt_carrier_fl_num+op_unique_carrier+tail_num+op_carrier_fl_num+origin+dest+crs_elapsed_time+distance+dayofyear+dayofweek+month+crs_dep_hr+crs_dep_min+crs_arr_hr+crs_arr_min+dep_time_day+arr_time_day+at_hub+origin_traffic+departure_traffic+historical_arr_delay+historical_dep_delay+historical_arr_delay_std+historical_arr_delay_75Q+historical_arr_delay_25Q',data=train).fit()
# multinomModel.summary()

The Durbin Watson statistic is a test for autocorrelation in a data set.
The DW statistic always has a value between zero and 4.0.
A value of 2.0 means there is no autocorrelation detected in the sample. Values from zero to 2.0 indicate positive autocorrelation and values from 2.0 to 4.0 indicate negative autocorrelation.

In [ ]:
def preprocess_df(df):
    df.drop(['tail_num',
            'dest_airport_id',
            'dup',
            'flights'], axis=1, inplace=True)
    
    df['fl_date'] = pd.to_datetime(df['fl_date'], format='%Y-%m-%d')
    df['month'] = df['fl_date'].dt.month
    df['day'] = df['fl_date'].dt.day
    df['weekday'] = df['fl_date'].dt.dayofweek
    
    # Drop fl_date, not needed anymore
    df.drop('fl_date', axis=1, inplace=True)
    
    # Get orgin/dest statename
    df['origin_state'] = df['origin_city_name'].apply(lambda x: x.split(', ')[1])
    df['dest_state'] = df['dest_city_name'].apply(lambda x: x.split(', ')[1])
    
    df.drop(['origin_city_name', 'dest_city_name'], axis=1, inplace=True)
    
    
    df['origin'] = df['origin'].astype('category')
    df['dest'] = df['dest'].astype('category')
    
    # Define new function to convert crs time
    def convert_time_to_string(x):
        if np.isnan(x):
            return '00:00'

        if x == 2400:
            return '23:59'

        string = str(int(x))

        if len(string) < 4:
            for _ in range(4-len(string)):
                string = '0' + string

        return string[:2] + ':' + string[2:]
    
    df['crs_dep_time'] = df['crs_dep_time'].apply(lambda x: convert_time_to_string(x))
    df['crs_arr_time'] = df['crs_arr_time'].apply(lambda x: convert_time_to_string(x))
    
    # Convert to datetimes
    df['crs_dep_time'] = pd.to_datetime(df['crs_dep_time'], format='%H:%M')  
    df['crs_arr_time'] = pd.to_datetime(df['crs_arr_time'], format='%H:%M')
    
    # Create hr column, minute col
    df['dep_hr'] = df['crs_dep_time'].dt.hour
    df['dep_min'] = df['crs_dep_time'].dt.minute   
    df['arr_hr'] = df['crs_arr_time'].dt.hour
    df['arr_min'] = df['crs_arr_time'].dt.minute                                                
    
    # Drop crs
    df.drop(['crs_dep_time', 'crs_arr_time'], axis=1, inplace=True)
    
    categorical = ['mkt_carrier', 'origin_state', 'dest_state']
    numerical = ['mkt_carrier_fl_num', 
                 'origin_airport_id', 
                 'crs_elapsed_time', 
                 'distance', 
                 'month', 
                 'day', 
                 'weekday',
                 'dep_hr', 
                 'dep_min',
                 'arr_hr',
                 'arr_min']
    
    ct = ColumnTransformer(transformers=[('cat', OneHotEncoder(), categorical), 
                                         ('num', StandardScaler(), numerical)], remainder='passthrough')
    
    X_trainformed = ct.fit_transform(df)
    
    
    return X_trainformed